In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

from skimage import io, img_as_float, img_as_ubyte
from skimage.transform import rescale, resize
from skimage.util import view_as_blocks

%load_ext autotime

In [2]:
def normalize_img_values(img):
    img = img - img.min()
    img = img / img.max()
    img = img_as_ubyte(img)
    return img

def check_normalized(img):
    try:
        assert img.shape == (256, 256, 3), 'Error in shape of a read image '+str(img.shape)
        assert img.dtype == np.uint8, 'Error in dtype of a read image '+str(img.dtype)
        assert img.max() == 255, 'Error in max value of a read image '+str(img.max())
        assert img.min() == 0, 'Error in min value of a read image '+str(img.min())
        
    except AssertionError:
        return normalize_img_values(img_as_float(img))
    
    return img

def check_normalized_patch(patch, patch_size=64):
    try:
        assert patch.shape == (patch_size, patch_size, 3), 'Error in shape of a read patch '+str(patch.shape)
        assert patch.dtype == np.uint8, 'Error in dtype of a read patch '+str(patch.dtype)
        assert patch.max() == 255, 'Error in max value of a read patch '+str(patch.max())
        assert patch.min() == 0, 'Error in min value of a read patch '+str(patch.min())
        
    except AssertionError:
        return normalize_img_values(img_as_float(patch))
    
    return patch

time: 3.29 ms


In [6]:
def generate_patches(img, patch_size=64):
    assert ((patch_size & (patch_size - 1)) == 0) and patch_size > 0,'Patch size must be a positive power of 2'
    assert patch_size < 256,'Patch size must be lower than 256'
    
    return view_as_blocks(img, block_shape=(patch_size, patch_size, 3))

def sliding_window(img, window_size=64, )

time: 854 µs


In [10]:
# CREATE A PATCHED DATASET FROM THE BARK DATASET

train_size = 0.8
patch_size = 64
dataset_path = '../Datasets/Bark-dataset/'
output_path = train_path = '../Datasets/Bark-Train-patches-'+str(patch_size)+'/'
test_path = '../Datasets/Bark-Test-patches-'+str(patch_size)+'/'

assert ((patch_size & (patch_size - 1)) == 0) and patch_size > 0,'Patch size must be a positive power of 2'
assert patch_size < 256,'Patch size must be lower than 256'

if not os.path.exists(train_path): os.mkdir(train_path)
if not os.path.exists(test_path): os.mkdir(test_path)

files = sorted(os.listdir(dataset_path), key=lambda x: int( (x.split('_')[1]).split('.')[0] ))
num_images = len(files)
patches_per_image = int(264/patch_size)**2
num_train_samples = int(num_images*0.8)*patches_per_image
num_test_samples = num_samples - num_train_samples

assert num_train_samples % patches_per_image == 0 and num_train_samples > 0,'Num train samples '+str(num_train_samples)+' not multiple of: '+str(patches_per_image)
assert num_test_samples % patches_per_image == 0 and num_test_samples > 0,'Num test samples '+str(num_test_samples)+' not multiple of: '+str(patches_per_image)

img_count = 0
patch_count = 0
for filename in files:
    if '.jpg' not in filename: continue
    
    img = io.imread(dataset_path+filename)
    #print(img.shape, img.dtype, img.max(), img.min())
    img = check_normalized(img)
    
    #print(np.reshape(generate_patches(img, patch_size), newshape=(-1, patch_size, patch_size, 3)).shape )
    raw_patches = generate_patches(img, patch_size)
    patches = np.reshape(raw_patches, (-1, patch_size, patch_size, 3))
    for patch in patches:
        patch = check_normalized_patch(patch)
        
        #plt.imshow(patch)
        #plt.show()
        
        assert patch.shape == (patch_size, patch_size, 3), 'Error in shape of a read image '+str(patch.shape)
        assert patch.dtype == np.uint8, 'Error in dtype of a read image '+str(patch.dtype)
        assert patch.max() == 255, 'Error in max value of a read image '+str(patch.max())
        assert patch.min() == 0, 'Error in min value of a read image '+str(patch.min())

        image_name = str(patch_count) + '.jpg'
        io.imsave(output_path + str(filename.split('_')[0]) + '_' + str(img_count) + '_' + image_name, patch)
        patch_count += 1
        if patch_count == num_train_samples: output_path = test_path
    
    img_count += 1
    
print(num_train_samples,'patches saved to',train_path,'from a total of',img_count,'images')
print(num_test_samples,'patches saved to',test_path,'from a total of',img_count,'images')

/home/albert.garcia/anaconda3/lib/python3.7/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


47728 patches saved to ../Datasets/Bark-Train-patches-64/ from a total of 3729 images
11936 patches saved to ../Datasets/Bark-Test-patches-64/ from a total of 3729 images
time: 1min 11s
